# Laboratorio #2: Redes Neuronales

Oscar Juárez - 17315

Inteligencia Artificial

Fecha: 06/04/2020

## Importar librerías útiles

In [1]:
# Librerías externas
import numpy as np
import pandas as pd
import pickle
import random
from functools import reduce
from scipy import optimize as op
import matplotlib.pyplot as plt

# Librerías locales
from fns_redes_neuronales import *

# 1. Lectura de datasets y definición de datos a utilizar

In [2]:
trainSet = pd.read_csv('data/fashion-mnist_train.csv')
testSet = pd.read_csv('data/fashion-mnist_test.csv')

In [3]:
NROWS = len(trainSet)
TRAIN_ROWS = int(NROWS * 0.6)
NCV = int(NROWS * 0.2)
random.seed(1234)

In [4]:
# Desordenamos nuestros datos
data = trainSet.iloc[:, :].values
# random.shuffle(data)
data = pd.DataFrame(data)

# Separamos datos del entrenamiento, cross validation y test.
train = data.iloc[:TRAIN_ROWS, :]
cv = data.iloc[TRAIN_ROWS : TRAIN_ROWS + NCV, :]
test = data.iloc[TRAIN_ROWS + NCV:, :]

# Normalizamos los datos a usar, sin la columna de la etiqueta
training = train.iloc[:, 1:] / 1000.0

# Obtenemos los labels correspondientes en un vector aparte
training_labels = train.iloc[:, 0].values
training_labels = training_labels.reshape(len(training_labels),1)

#### Hacemos vector de respuestas con formato correcto

In [5]:
temp = np.zeros((len(training_labels),10)).astype(int)

for i in range(temp.shape[0]):
    pos = training_labels[i]
    a = np.zeros(10)
    a[pos] = 1
    temp[i] = a

training_labels = temp
print(training_labels)

[[0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


# 2. Definir la red neuronal

In [6]:
m,n = training.shape

# Arquitectura de la red
ARCH = np.array([
    n,
    130,
    10
])

# Shape del vector de thetas
theta_shapes = np.hstack(
    (
        ARCH[1:].reshape(len(ARCH) - 1, 1),
        (ARCH[:-1] + 1).reshape(len(ARCH) - 1, 1)
    )
)

# Se convierte la lista de matrices a una lista
flat_thetas = flatten_list_of_arrays(
    [np.random.rand(*theta_shape) for theta_shape in theta_shapes]
)

In [7]:
# Vector de thetas inicial, el módelo aún no está entrenado
flat_thetas

array([0.03429105, 0.98317641, 0.40740801, ..., 0.24585098, 0.37702133,
       0.96389113])

# 3. Optimizar la función de costo para entrenar al modelo

##### IMPORTANTE: Saltar este paso si no se creará el modelo desde 0

In [ ]:
result = op.minimize(
    fun = cost_function,
    x0 = flat_thetas,
    args = (theta_shapes, training, training_labels),
    method = 'L-BFGS-B',
    jac = back_propagation,
    options = {'disp': True, 'maxiter': 3000}
)

## Guardar thetas optimizadas en un archivo con pickle

##### IMPORTANTE: Saltar este paso si el modelo no fue entrenado desde 0

In [ ]:
file = open("trained_model.txt","wb")
pickle.dump(result.x, file)
file.close()

## Extraer thetas optimizadas de modelo previamente entrenado

In [8]:
with (open("trained_model.txt", "rb")) as openfile:
    while True:
        try: optimized_thetas = pickle.load(openfile)
        except EOFError: break

optimized_thetas = np.asarray(optimized_thetas)

# 4. Predicción de datos

#### Predicción sobre cross validation

In [9]:
# Se define la data a utilizar de manera correcta
cv_data = cv.iloc[:, 1:] / 1000.0

# Obtener respuestas por aparte
cv_labels = np.asarray(cv.iloc[:, 0])
cv_labels = cv_labels.reshape(len(cv_data), 1)
cv_labels = (cv_labels == np.array(range(10))).astype(int)

cv_labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [10]:
# Hacer predicción en base a datos
Thetas = inflate_matrixes(optimized_thetas, theta_shapes)
a = feed_forward(Thetas, cv_data)

results = [a[-1], cv_labels]

#### Resultados sobre Cross Validation

In [11]:
correct, incorrect = 0,0

for predictions, answers in zip(results[0], results[1]):
    pred = np.where(predictions == np.amax(predictions))
    ans = np.where(answers == np.amax(answers))
    
    if pred == ans: correct = correct + 1
    else: incorrect = incorrect + 1

results_string = """
TOTAL DE ELEMENTOS: {}
PREDICCIONES ACERTADAS: {}
PREDICCIONES INCORRECTAS: {}
RENDIMIENTO: {}%
""".format(
    len(cv_data),
    correct,
    incorrect,
    round(100 * correct/(correct + incorrect), 2)
)

print(results_string)


TOTAL DE ELEMENTOS: 12000
PREDICCIONES ACERTADAS: 10167
PREDICCIONES INCORRECTAS: 1833
RENDIMIENTO: 84.72%

